In [1]:
import pandas as pd
import netCDF4 as nc
import numpy as np
import cv2
import os
import errno
import geopandas as gpd
import seaborn as sns
import shapely
from shapely.geometry import shape, Polygon, Point, MultiPoint, box, LineString
import xarray as xr
from sklearn.linear_model import LinearRegression
from pyproj import Geod
from shapely import wkt

In [28]:
def polyarc(lon,lat,r1,r2,r3,r4):
    ctpos = [lon,lat]
    u=111.1
    arc = arco(r1/u,r2/u,r3/u,r4/u,ctpos)
    polyar = Polygon(list(zip(arc[0],arc[1])))
    return polyar
def polfun(x):
    return polyarc(x['lon'],x['lat'],x['TCOR'],x['TCOR'],x['TCOR'],x['TCOR'])
def arco(r1,r2,r3,r4,c):
    ang = [np.arange(0,np.pi/2,0.01), np.arange(np.pi/2,np.pi,0.01),
           np.arange(np.pi,(3/2)*np.pi,0.01),np.arange((3/2)*np.pi,2*np.pi,0.01)]
    grafx, grafy = [], []
    for i,r in enumerate([r1,r2,r3,r4]):
        grafx.append(r*np.cos(ang[i])+c[0])
        grafy.append(r*np.sin(ang[i])+c[1])
    return (np.concatenate(grafx, axis = 0),np.concatenate(grafy, axis = 0))
def dateind(hh,dd,mm,yy):
    suma = 0
    if (yy == np.arange(2000,2021,4)).any():
        mes = [744,696,744,720,744,720,744,744,720,744,720,744]
    else:
        mes = [744,672,744,720,744,720,744,744,720,744,720,744]

    for i in np.arange(1,mm):
        suma = (suma+mes[i-1])
    suma = suma+(dd-1)*24+hh
    return suma


def areapoly(geom):
    geod = Geod(ellps="WGS84")
    area = abs(geod.geometry_area_perimeter(geom)[0])/1e+6
    return area

In [37]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry
import xarray as xr

def cargar_datos_climaticos(yy):
    """
    Carga los datos climáticos para un año específico.

    Parámetros:
    yy (int): Año para el cual se cargan los datos.

    Retorno:
    dict: Contiene los datasets de humedad, cizalladura, divergencia, velocidad vertical y vorticidad.
    """
    hum = xr.open_dataset(f'D:/hum/hum{yy}.nc')
    wshr = xr.open_dataset(f'D:/ws/ws{yy}.nc')
    div = xr.open_dataset(f'D:/divergence/velver{yy}.nc')
    vver = xr.open_dataset(f'D:/vver/velver{yy}.nc')
    vor = xr.open_dataset(f'D:/vor/vor{yy}.nc')
    
    return {'hum': hum, 'wshr': wshr, 'div': div, 'vver': vver, 'vor': vor}

def extraer_valores_por_tiempo(datos, time):
    """
    Extrae los valores de los datasets climáticos para un tiempo específico.

    Parámetros:
    datos (dict): Datasets climáticos.
    time (int): Índice temporal para el cual se extraen los datos.

    Retorno:
    dict: Contiene los valores de humedad, cizalladura, divergencia, velocidad vertical y vorticidad.
    """
    hum_val = datos['hum'].r[time].data
    ciz_val = datos['wshr'].value[time].data
    div_val = datos['div'].d[time].data
    vvr_val = datos['vver'].w[time].data
    vor_val = datos['vor'].vo[time].data
    
    return {'hum': hum_val, 'ciz': ciz_val, 'div': div_val, 'vvr': vvr_val, 'vor': vor_val}

def crear_dataframe_climatico(lat, lon, valores):
    """
    Crea un DataFrame a partir de los valores climáticos filtrados.

    Parámetros:
    lat, lon (arrays): Coordenadas latitudinales y longitudinales.
    valores (dict): Valores climáticos para las coordenadas.

    Retorno:
    DataFrame: Contiene los datos climáticos organizados por latitud y longitud.
    """
    x, y = np.meshgrid(lon, lat)
    indx = np.where(valores['hum'] != -9999)  # Filtra los valores no válidos
    indxx, indxy = x[indx], y[indx]
    
    df = pd.DataFrame({
        'lon': indxx, 
        'lat': indxy, 
        'hum': valores['hum'][indx], 
        'ciz': valores['ciz'][indx], 
        'div': valores['div'][indx], 
        'vvr': valores['vvr'][indx], 
        'vor': valores['vor'][indx]
    })
    
    return df

def filtrar_por_radio(df, ctpos, R_max, i):
    """
    Filtra los datos en función de un radio alrededor de una posición central.

    Parámetros:
    df (DataFrame): DataFrame de valores climáticos.
    ctpos (tuple): Posición central (longitud, latitud).
    R_max (Series): Radio máximo para cada punto.
    i (int): Índice actual.

    Retorno:
    DataFrame: DataFrame filtrado según el radio.
    """
    factor = R_max.iloc[i] / 111.1
    f1 = (ctpos[1] - factor - 1 < df.lat) & (df.lat < ctpos[1] + factor + 1)
    f2 = (ctpos[0] - factor - 1 < df.lon) & (df.lon < ctpos[0] + factor + 1)
    return df.loc[f1 & f2]

def calcular_promedios(dfSP, alex, i):
    """
    Calcula los promedios de los valores climáticos y los asigna al DataFrame original.

    Parámetros:
    dfSP (GeoDataFrame): DataFrame geoespacial filtrado.
    alex (DataFrame): DataFrame original con los datos de entrada.
    i (int): Índice actual.

    Retorno:
    None: Los valores son actualizados directamente en el DataFrame `alex`.
    """
    alex.loc[i, 'hum'] = dfSP['hum'].mean()
    alex.loc[i, 'ciz'] = dfSP['ciz'].mean()
    alex.loc[i, 'div'] = dfSP['div'].mean()
    alex.loc[i, 'vvr'] = dfSP['vvr'].mean()
    alex.loc[i, 'vor'] = dfSP['vor'].mean()

def var_env_mean(alex):
    """
    Función principal que realiza el cálculo de los valores promedio del entorno.
    """
    geometry = alex.apply(polfun, axis=1)  # Aplicar función de polígono para generar geometrías
    alex['geometry'] = geometry  # Añadir geometría al DataFrame

    for i in range(len(alex)):
        # Obtener datos específicos por fila
        hh = alex.hh.iloc[i]
        yy = alex.yy.iloc[i]
        dd = alex.dd.iloc[i]
        mm = alex.mm.iloc[i]

        # Cargar datos climáticos
        datos = cargar_datos_climaticos(yy)
        lat = datos['hum'].latitude.data
        lon = datos['hum'].longitude.data

        # Obtener el índice temporal
        time = dateind(hh, dd, mm, yy)
        
        # Extraer los valores climáticos
        valores = extraer_valores_por_tiempo(datos, time)
        
        # Crear DataFrame con los valores extraídos
        df = crear_dataframe_climatico(lat, lon, valores)
        
        # Posición central y filtrado por radio
        ctpos = (alex.lon.iloc[i], alex.lat.iloc[i])
        R_max = alex.loc[:, ['TCOR']].max(axis=1)
        df = filtrar_por_radio(df, ctpos, R_max, i)
        
        # Convertir DataFrame a GeoDataFrame
        dfSP = gpd.GeoDataFrame(df, geometry=[shapely.geometry.Point(xy) for xy in zip(df.lon, df.lat)])
        
        # Filtrar puntos dentro del polígono
        ipn = dfSP['geometry'].intersects(alex['geometry'][i])
        dfSP = dfSP.loc[ipn].reset_index(drop=True)
        
        # Calcular promedios y actualizar valores en el DataFrame original
        calcular_promedios(dfSP, alex, i)
        
        print(f'Procesado: {i+1}/{len(alex)}')
    
    # Calcular área del polígono
    alex['areap'] = alex['geometry'].apply(areapoly)
    
    # Retornar DataFrame sin la columna de geometría
    return alex.drop(['geometry'], axis=1).reset_index(drop=True)


In [30]:
names = ['ct','date', 'time', 'lat', 'lon', 'MWS', 'CPSL', 'ERMWS', 'R34', 'R50', 'R64', 'R100', 'TCOR']
alex = pd.read_csv('../NA_PA.dat', sep="\t", skip_blank_lines=True, header = None, names = names)

In [31]:
# Crear nuevas columnas para año, mes, día
# Separar la columna fecha en año, mes, día y tiempo
alex['date'] = pd.to_datetime(alex['date'])
alex['yy'] = alex['date'].dt.year
alex['mm'] = alex['date'].dt.month
alex['dd'] = alex['date'].dt.day

In [32]:
alex['hh'] = pd.to_datetime(alex['time'], format='%H:%M').dt.hour

In [33]:
df = alex.head(15).copy()

In [34]:
df

,ct,date,time,lat,lon,MWS,CPSL,ERMWS,R34,R50,R64,R100,TCOR,yy,mm,dd,hh
0,AL012000,2000-06-07,18:00,21.0,-93.0,46.25,1008,54.45,NaN,NaN,NaN,NaN,673.5,2000,6,7,18
1,AL012000,2000-06-08,00:00,20.9,-92.8,46.25,1009,54.36,NaN,NaN,NaN,NaN,673.0,2000,6,8,0
2,AL012000,2000-06-08,06:00,20.7,-93.1,46.25,1010,54.17,NaN,NaN,NaN,NaN,671.5,2000,6,8,6
3,AL012000,2000-06-08,12:00,20.8,-93.5,46.25,1010,54.27,NaN,NaN,NaN,NaN,672.0,2000,6,8,12
4,AL022000,2000-06-23,00:00,9.5,-19.8,46.25,1010,44.83,NaN,NaN,NaN,NaN,598.0,2000,6,23,0
5,AL022000,2000-06-23,06:00,9.6,-21.0,55.50,1009,43.15,NaN,NaN,NaN,NaN,639.0,2000,6,23,6
6,AL022000,2000-06-23,12:00,9.9,-22.6,55.50,1009,43.37,NaN,NaN,NaN,NaN,641.0,2000,6,23,12
7,AL022000,2000-06-23,18:00,10.2,-24.5,55.50,1009,43.59,NaN,NaN,NaN,NaN,643.0,2000,6,23,18
8,AL022000,2000-06-24,00:00,10.1,-26.2,55.50,1009,43.52,NaN,NaN,NaN,NaN,642.0,2000,6,24,0
9,AL022000,2000-06-24,06:00,9.9,-27.8,55.50,1009,43.37,NaN,NaN,NaN,NaN,641.0,2000,6,24,6


In [39]:
def ejecutar_var_env_mean(alex):
    """
    Ejecuta en orden las funciones necesarias para calcular los promedios ambientales
    en el DataFrame `alex`.

    Parámetros:
    alex (DataFrame): DataFrame con los datos iniciales.

    Retorno:
    DataFrame: DataFrame actualizado con los valores promedios calculados.
    """
    # Aplicar la función `var_env_mean` que integra todo el flujo
    resultado = var_env_mean(alex)
    return resultado



In [41]:
# Suponiendo que `alex` ya está definido y cargado con datos
resultado = ejecutar_var_env_mean(alex)

# Imprimir o guardar el resultado
resultado.to_csv('env_na_tcor.dat', sep = '\t', index = False, header = False)

Procesado: 1/11132
Procesado: 2/11132
Procesado: 3/11132
Procesado: 4/11132
Procesado: 5/11132
Procesado: 6/11132
Procesado: 7/11132
Procesado: 8/11132
Procesado: 9/11132
Procesado: 10/11132
Procesado: 11/11132
Procesado: 12/11132
Procesado: 13/11132
Procesado: 14/11132
Procesado: 15/11132
Procesado: 16/11132
Procesado: 17/11132
Procesado: 18/11132
Procesado: 19/11132
Procesado: 20/11132
Procesado: 21/11132
Procesado: 22/11132
Procesado: 23/11132
Procesado: 24/11132
Procesado: 25/11132
Procesado: 26/11132
Procesado: 27/11132
Procesado: 28/11132
Procesado: 29/11132
Procesado: 30/11132
Procesado: 31/11132
Procesado: 32/11132
Procesado: 33/11132
Procesado: 34/11132
Procesado: 35/11132
Procesado: 36/11132
Procesado: 37/11132
Procesado: 38/11132
Procesado: 39/11132
Procesado: 40/11132
Procesado: 41/11132
Procesado: 42/11132
Procesado: 43/11132
Procesado: 44/11132
Procesado: 45/11132
Procesado: 46/11132
Procesado: 47/11132
Procesado: 48/11132
Procesado: 49/11132
Procesado: 50/11132
Procesado